In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

# DATA

In [2]:
train_df = pd.read_csv("../data/travel_insurance_prediction_train.csv")
test_df = pd.read_csv("../data/travel_insurance_prediction_test.csv")

In [3]:
X_cat = train_df.drop(["Customer","TravelInsurance"],axis=1).select_dtypes(include='object').columns
X_num = train_df.drop(["Customer","TravelInsurance"],axis=1).select_dtypes(include='int64').columns
X_scaler = ["AnnualIncome","Age","FamilyMembers"]
X_cat.tolist(), X_num.tolist()

(['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad'],
 ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases'])

In [4]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
encoder = OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore")
scaler = MinMaxScaler()
transformer = make_column_transformer((encoder, X_cat),(scaler,X_scaler),remainder="passthrough")
#train, validation = train_test_split(train_df)
X = transformer.fit_transform(train_df.drop(columns=["Customer", "TravelInsurance"]))
y = train_df["TravelInsurance"].values
X_test = transformer.transform(test_df.drop(columns=["Customer"]))

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=2, stratify=y)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()
param = {
    'n_estimators':[50],
    'criterion':['gini'],
    'max_depth':[25,20,15,10],
    'min_samples_split':[2,5,10,15,20]
}
cv = GridSearchCV(estimator=rfc,param_grid=param, scoring='f1',n_jobs=-1,cv=5)
cv.fit(X_train, y_train)
from sklearn.metrics import classification_report
print(classification_report(y_train, cv.predict(X_train)))
print(classification_report(y_val, cv.predict(X_val)))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92       718
           1       0.93      0.75      0.83       399

    accuracy                           0.89      1117
   macro avg       0.90      0.86      0.87      1117
weighted avg       0.89      0.89      0.89      1117

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       240
           1       0.76      0.70      0.73       133

    accuracy                           0.81       373
   macro avg       0.80      0.79      0.79       373
weighted avg       0.81      0.81      0.81       373



# OUTPUT

In [ ]:
test_id = test_df["Customer"]
test_pred = best_tree_clf.predict(X_test)

In [ ]:
submission = pd.DataFrame(list(zip(test_id, test_pred)), columns=["Customer", "TravelInsurance"])

In [ ]:
submission.head()

In [ ]:
submission.to_csv("../data/travel_insurance_submission-try-2.csv", header=True, index=False)